In [1]:
# class Ademonla:
#     def __init__(self, scoring_function = 'f1', n_iter = 50):
#         self.scoring_function = scoring_function
#         self.n_iter = n_iter
    
#     def fit(self):

In [2]:
def Full_course(Trainfile, Testfile, target, ptype, metric):
    # importing all required libraries
    import pandas as pd
    import numpy as np
    
    # from sklearn.model_selection import RandomizedSearchCV
    from sklearn.pipeline import Pipeline
    from imblearn.over_sampling import SMOTE

    import warnings
    warnings.filterwarnings('ignore')
    
    # Loading the dataframe
    Trainframe = pd.read_csv(Trainfile,encoding='cp1252')
    Testframe = pd.read_csv(Testfile,encoding='cp1252')

    # Removes columns with more than 70% missing values
    crazy = [i for i in Trainframe.columns if Trainframe[i].isna().sum() >= 0.7*len(Trainframe[i])]
    Trainframe = Trainframe.drop(crazy, axis=1)
    Testframe = Testframe.drop(crazy, axis=1)

   # Sorting based on datatypes
    date = [d for d in Trainframe.columns if 'Date' in d]
    cat = [n for n in Trainframe.columns if Trainframe[n].dtype == object and n not in date]
    # cat = []
    # for n in Trainframe.columns:
    #     if Trainframe[n].dtype == object or Trainframe[n].dtype is 'category' or Trainframe[n].dtype == bool and n not in date:
    #         cat.append(n)
    num = [p for p in Trainframe.columns if p not in cat]

    # filling up missing values
    imp = KNNImputer(n_neighbors=5)
    missing_num = [i for i in num if Trainframe[i].isna().sum() != 0]
    if missing_num != []:
        Trainframe[missing_num] = imp.fit_transform(Trainframe[missing_num])
        Testframe[missing_num] = imp.transform(Testframe[missing_num])

    obj = SimpleImputer(missing_values = np.nan, strategy='most_frequent')
    missing_cat = [i for i in cat if Trainframe[i].isna().sum() != 0]
    if missing_cat != []:
        Trainframe[missing_cat] = obj.fit_transform(Trainframe[missing_cat])
        Testframe[missing_cat] = obj.transform(Testframe[missing_cat])

        Trainframe[cat] = Trainframe[cat].astype('category')
        Testframe[cat] = Testframe[cat].astype('category')

    # Dealing with date features
    if date != []:
        Trainframe[date] = obj.fit_transform(Trainframe[date])
        Testframe[date] = obj.transform(Testframe[date])
        for i in date:
            Trainframe[i] = pd.to_datetime(Trainframe[i]).astype('category')
            Trainframe[f'{i}_months'] = Trainframe[i].dt.month.astype('category')
            Trainframe[f'{i}_days'] = Trainframe[i].dt.day.astype('category')
            Trainframe[f'{i}_years'] = Trainframe[i].dt.year.astype('category')

            Testframe[i] = pd.to_datetime(Testframe[i]).astype('category')
            Testframe[f'{i}_months'] = Testframe[i].dt.month.astype('category')
            Testframe[f'{i}_days'] = Testframe[i].dt.day.astype('category')
            Testframe[f'{i}_years'] = Testframe[i].dt.year.astype('category')

        Trainframe = Trainframe.drop(date, axis = 1)
        Testframe = Testframe.drop(date, axis = 1)
    
    iid = Testframe['ID']

    # Removing columns with high variance
    uniq = [i for i in cat if Trainframe[i].nunique() >= 0.7 * Trainframe.shape[0]]
    Trainframe = Trainframe.drop(uniq, axis=1)
    Testframe = Testframe.drop(uniq, axis=1)

    Testframe1 = Testframe.copy()

    X = Trainframe.drop(target, axis =1)
    y = Trainframe[target]

    # Train and test split
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 101)
    trainX, testX, trainy, testy = train_test_split(X, y, test_size = 0.2, random_state = 101)
    t = [i for i in cat if i not in uniq]


    #Selecting Classification or Regression---TPOT
    if ptype == 'class':
      # Encoding
        encode = ce.BinaryEncoder(t)
        train_X = encode.fit_transform(train_X)
        test_X = encode.transform(test_X)
        Testframe = encode.transform(Testframe)
    
      # Resampling
        oversample = SMOTE()
        train_X, train_y = oversample.fit_resample(train_X, train_y)
        #test_X, test_y = oversample.fit_resample(test_X, test_y)

        # Feature Selection 
        if len(train_X.columns) > 20:
            dec = PCA(0.98)
            train_X = dec.fit_transform(train_X)
            test_X = dec.transform(test_X)
            Testframe = dec.transform(Testframe)

      # Standardization
        sc = RobustScaler()
        train_X = sc.fit_transform(train_X)
        test_X = sc.transform(test_X)
        Testframe = sc.transform(Testframe)
        
      # Using the TPOT AutoML
        tpot_class = TPOTClassifier(max_time_mins=10,scoring= metric)
        tpot_class.fit(train_X, train_y) #fit the model on the Train

        ypred = tpot_class.predict(test_X)
        # Predict the Testfile
        ypred1 = tpot_class.predict(Testframe)
        # Check the Scores
        sol = tpot_class.score(train_X, train_y)
        sol1 = tpot_class.score(test_X, test_y)        

    elif ptype == 'reg':
        # Encoding the categorical Variables
        encode = ce.TargetEncoder(t, smoothing=8, min_samples_leaf=5)
        train_X = encode.fit_transform(train_X, train_y)
        test_X = encode.transform(test_X)
        Testframe = encode.transform(Testframe)

      # Standardization
        sc = RobustScaler()
        train_X = sc.fit_transform(train_X)
        test_X = sc.transform(test_X)
        Testframe = sc.transform(Testframe)

        # Using the TPOT AutoML
        tpot_reg = TPOTRegressor(max_time_mins=10,scoring= metric)
        tpot_reg.fit(train_X, train_y) # fit the model in the Train set

        ypred = tpot_reg.predict(test_X)
        # Predict the Test file
        ypred1 = tpot_reg.predict(Testframe)
        # Check the Scores
        sol = tpot_reg.score(train_X, train_y)    
        sol1 = tpot_reg.score(test_X, test_y)
    
    # Outcomes
    alg = pd.DataFrame({"ID":iid,f'{target}':ypred1})
    Scores = f'TPOT: The Train score is {sol}, The Test score is {sol1}'
    print(Scores)



    #Selecting Classification or Regression--- Ensemble
    if ptype == 'class':
        # category encoding 
        encode = ce.BinaryEncoder(t)
        trainX = encode.fit_transform(trainX)
        testX = encode.transform(testX)
        Testframe1 = encode.transform(Testframe1)

        # Resampling
        oversample = SMOTE()
        trainX, trainy = oversample.fit_resample(trainX, trainy)
        #testX, testy = oversample.fit_resample(testX, testy)
        
        # Feature Selection 
        if len(trainX.columns) > 20:
            dec = PCA(0.98)
            trainX = dec.fit_transform(trainX)
            testX = dec.transform(testX)
            Testframe1 = dec.transform(Testframe1)

        # Standardization
        sc = RobustScaler()
        trainX = sc.fit_transform(trainX)
        testX = sc.transform(testX)
        Testframe1 = sc.transform(Testframe1)

        # Modeling
        # Finding the BaseLine perfomance of the various models
        models = []

        # Adding algorthms
        models.append(('cat', CatBoostClassifier(verbose=0)))
        models.append(('mlp', MLPClassifier(verbose=0)))
        models.append(('lgr', LogisticRegression(verbose=0)))
        models.append(('svc', SVC(probability=True)))
        models.append(('knc', KNeighborsClassifier()))
        models.append(('lgbm', LGBMClassifier()))
        models.append(('sgd', SGDClassifier()))
        models.append(('ext', ExtraTreesClassifier()))
        models.append(('rfc', RandomForestClassifier()))
        models.append(('gbc', GradientBoostingClassifier()))
        models.append(('xgb', XGBClassifier()))

        # evaluate -cross validation- each model in turn
        results = []
        mods = []
        names = []
        scoring =['roc_auc', metric]
        for name, model in models:
            kfold = StratifiedKFold(n_splits=7, shuffle= True)
            cv_results = cross_validate(model, trainX, trainy, cv=kfold, scoring=scoring, return_train_score=True)
            results.append([cv_results['test_roc_auc'].mean(), cv_results[f'test_{metric}'].mean()])
            mods.append(model)
            names.append(name)
        alg = pd.DataFrame({'models': models, 'roc_auc':[m[0] for m in results] , metric: [m[1] for m in results]})
        # Ensemble
        vtc = VotingClassifier([n for n in alg.sort_values(metric, ascending=False)['models'][:2]], weights =[i for i in alg[metric].nlargest(2)], voting='soft')
        vtc.fit(trainX, trainy) # fit the model in the Train set
        # if vote < alg.metric.max():
        #   alg.metric.max()['models']
        ypred = vtc.predict(testX)
        
        # Predict the Test file
        ypred1 = vtc.predict(Testframe1)
        ypredp1 = vtc.predict_proba(Testframe1)
        
        # Check the Scores
        sol = vtc.score(trainX, trainy)    
        sol1 = vtc.score(testX, testy)

    elif ptype == 'reg':
        # category encoding 
        encode = ce.TargetEncoder(cols=t, smoothing=8, min_samples_leaf=5)
        trainX = encode.fit_transform(trainX, trainy)
        testX = encode.transform(testX)
        Testframe1 = encode.transform(Testframe1)

        # Feature Selection 
        dec = PCA(0.98)
        trainX = dec.fit_transform(trainX)
        testX = dec.transform(testX)
        Testframe1 = dec.transform(Testframe1)

        # Standardization
        sc = RobustScaler()
        trainX = sc.fit_transform(trainX)
        testX = sc.transform(testX)
        Testframe1 = sc.transform(Testframe1)

        # Modeling
        # Finding the BaseLine perfomance of the various models

        # Prepare models
        models = []

        # Adding algorthms
        models.append(('cat', CatBoostRegressor(verbose=0)))
        models.append(('mlp', MLPRegressor(verbose=0)))
        models.append(('lgr', LinearRegression()))
        models.append(('lss', Lasso()))
        models.append(('eln', ElasticNet()))
        models.append(('svc', SVR()))
        models.append(('knc', KNeighborsRegressor()))
        models.append(('sgd', SGDRegressor()))
        models.append(('lgbm', LGBMRegressor()))
        models.append(('ext', ExtraTreesRegressor()))
        models.append(('rfc', RandomForestRegressor()))
        models.append(('gbc', GradientBoostingRegressor()))
        models.append(('xgb', XGBRegressor()))
       
        # evaluate -cross validation- each model in turn
        results = []
        names = []
        scoring =['neg_root_mean_squared_error', metric]
        for name, model in models:
            kfold = KFold(n_splits=7, shuffle= True)
            cv_results = cross_validate(model, trainX, trainy, cv=kfold, scoring=scoring, return_train_score=True)
            results.append([-cv_results['test_neg_root_mean_squared_error'].mean(),cv_results[f'test_{metric}'].mean()])
            names.append(name)
        alg = pd.DataFrame({'models': models, 'RMSE':[m[0] for m in results], metric:[m[1] for m in results]})
        # Ensemble
        vtr = VotingRegressor([n for n in alg.sort_values('RMSE', ascending=True)['models']][:2], weights =[i for i in alg[metric].nsmallest(2)], voting = 'hard')
        vtr.fit(trainX, trainy) # fit the model in the Train set

        ypred = vtr.predict(test_X)
        # Predict the Test file
        ypred1 = vt.predict(Testframe1)
        # Check the Scores
        sol = vtr.score(trainX, trainy)    
        sol1 = vtr.score(testX, testy)

    # Outcomes
    alg = pd.DataFrame({'ID':iid ,f'{target}':ypred1})
    Scores = f'Ensemble: The Train score is {sol}, The Test score is {sol1}'
    print(Scores)


    return alg.head()

In [3]:
Demon('C:/Users/iDAFAdmin/Documents/Projects/Zindi/Autoland Insurance Claim/Train.csv', 'C:/Users/iDAFAdmin/Documents/Projects/Zindi/Autoland Insurance Claim/Test.csv', 'target', 'class', 'f1')

['Policy Start Date', 'Policy End Date', 'First Transaction Date']


KeyError: 'Policy Start Date'